In [42]:
import requests
from bs4 import BeautifulSoup


base_url = 'https://gameofthrones.fandom.com/wiki/Stormborn'
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, 'html.parser')
# Find the links on the page
dumps = [a['href'] for a in soup_index.find_all('a') if 
         a.has_attr('href')]


In [ ]:
import json
with open('characters.json') as json_file:
    data = json.load(json_file)
    for p in data['characters']:
        print('Name: ' + p['characterName'])
        print('')

In [38]:
CharacterDf = pd.read_json(r'characters.json')
CharacterDf.characters[3]

{'characterName': 'Aerys II Targaryen',
 'houseName': 'Targaryen',
 'characterImageThumb': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMWQzOWViN2ItNDZhOS00MmZlLTkxZTYtZDg5NGUwMGRmYWZjL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyMjk3NTUyOTc@._V1._SX100_SY140_.jpg',
 'characterImageFull': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMWQzOWViN2ItNDZhOS00MmZlLTkxZTYtZDg5NGUwMGRmYWZjL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyMjk3NTUyOTc@._V1_.jpg',
 'characterLink': '/character/ch0541362/',
 'actorName': 'David Rintoul',
 'actorLink': '/name/nm0727778/',
 'nickname': 'The Mad King',
 'royal': True,
 'killed': ['Brandon Stark', 'Rickard Stark'],
 'servedBy': ['Arthur Dayne', 'Jaime Lannister'],
 'parentOf': ['Daenerys Targaryen', 'Rhaegar Targaryen', 'Viserys Targaryen'],
 'siblings': ['Rhaella Targaryen'],
 'marriedEngaged': ['Rhaella Targaryen'],
 'killedBy': ['Jaime Lannister']}